# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from z3 import *
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import sys
import warnings
warnings.filterwarnings("ignore")
import json

# Clustering

In [2]:
# returns dbscan clusters
def dbscan(X, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    cluster = db.fit(X)
    #y_pred = db.fit_predict(X)
    #plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
    #plt.title("DBSCAN")
    #print("noise", sum(cluster.labels_ ==-1), "regular", sum(cluster.labels_ !=-1))
    #print(np.unique(cluster.labels_))
    return cluster

# Convex Hull

In [3]:
# returns convex hull of points associated with zones
def convex_hull(zone, points):
        '''
        Parameters
        ----------
        points : Array
            A set of points.
            
        Returns
        -------
        vertices : Points
            Arranges convex hull vertices in counter-clockwise orientation.
        '''    
        
        hull = ConvexHull(points)
        
        
        #plt.plot(points[:,0], points[:,1], 'o')
        #plt.ylim(0,120)
        #for simplex in hull.simplices:
        #    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
            
        simplices = hull.simplices
        
        vertices = []
        for index in hull.vertices:
            vertices.append((points[index][0], zone, points[index][1]))
        vertices.append((points[hull.vertices[0]][0], zone, points[hull.vertices[0]][1]))
        
        return vertices

# Cluster Boundary Acquisition

In [4]:
def get_cluster(dataframe, column, eps, min_samples):
    list_cluster = []
    for zone in range(num_zones):
        #plt.figure()
        mod_dataframe = pd.DataFrame()

        for i in range(len(dataframe)):
            if int(dataframe[column][i] == zone):
                mod_dataframe = mod_dataframe.append(dataframe.loc[i, ['Start Time', 'Duration']])

        features = np.empty([len(mod_dataframe), 2])
        features[:, 0:1] = mod_dataframe.loc[:, ['Start Time']].values
        features[:, 1:] = mod_dataframe.loc[:, ['Duration']].values
        cluster_model = dbscan(features, eps, min_samples)
        labels = cluster_model.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        for cluster in range(n_clusters):
            points = []
            for k in range(len(labels)):
                if labels[k] == cluster:
                    points.append(features[k])
            points = np.array(points)

            if len(points) >= 3:
                vertices = convex_hull(zone, points)
            else:
                print("zone", zone, "Problem Found!")

            list_cluster.append({"zone_id": zone, "cluster_id": cluster, "points": vertices})

    return list_cluster

# Clustering for House-A and Occupant-1

In [5]:
cleaned_duration_dataframe_house_A_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Duration-Dataframe_House-A_Occ-1.csv")
cleaned_duration_dataframe_house_A_occ_1
num_zones = 5
clusters_house_A_occ_1 = get_cluster(cleaned_duration_dataframe_house_A_occ_1, "Occupant-1 Zone ID", 20, 3)
# saving clusters to file
with open('../../data/shatter/Clusters-House_A_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_1))

# Clustering for House-A and Occupant-2

In [6]:
cleaned_duration_dataframe_house_A_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Duration-Dataframe_House-A_Occ-2.csv")
cleaned_duration_dataframe_house_A_occ_2
clusters_house_A_occ_2 = get_cluster(cleaned_duration_dataframe_house_A_occ_2, "Occupant-2 Zone ID", 20, 3)
# saving clusters to file
with open('../../data/shatter/Clusters-House_A_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_2))

# Clustering for House-B and Occupant-1

In [7]:
cleaned_duration_dataframe_house_B_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Duration-Dataframe_House-B_Occ-1.csv")
cleaned_duration_dataframe_house_B_occ_1
clusters_house_B_occ_1 = get_cluster(cleaned_duration_dataframe_house_B_occ_1, "Occupant-1 Zone ID", 30, 3)
# saving clusters to file
with open('../../data/shatter/Clusters-House_B_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_1))

# Clustering for House-B and Occupant-2

In [8]:
cleaned_duration_dataframe_house_B_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Duration-Dataframe_House-B_Occ-2.csv")
cleaned_duration_dataframe_house_B_occ_2
clusters_house_B_occ_2 = get_cluster(cleaned_duration_dataframe_house_B_occ_2, "Occupant-2 Zone ID", 30, 3)
# saving clusters to file
with open('../../data/shatter/Clusters-House_B_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_2))

# Range Claculation Function

In [9]:
def is_left( x, y, init_x, init_y, final_x, final_y):
     return ((final_x - init_x)*(y - init_y) - (final_y - init_y)*(x - init_x)) >= 0

def range_calculation(list_cluster):
    num_timeslots = 1440
    list_time_min = [[[] for j in range(num_timeslots)] for i in range(num_zones + 1)]
    list_time_max = [[[] for j in range(num_timeslots)] for i in range(num_zones + 1)]

    for i in range(len(list_cluster)):
        zone_id = list_cluster[i]["zone_id"]
        min_x_range = 1440
        max_x_range = 0

        ##################################################################
        ##################### Zone Constraints ###########################
        ##################################################################
        x = Int('x')
        y = Int('y')

        points = list_cluster[i]["points"]
        zone_constraints = []

        and_constraints = []
        for j in range(len(points) - 1):
            and_constraints.append(is_left(x, y, points[j][0], points[j][2], points[j + 1][0], points[j + 1][2]))

        zone_constraints.append(And(and_constraints))

        #print(zone_constraints)
        ####### Minimum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.minimize(x)
        o.check()
        
        min_x_range = int(str(o.model()[x]))

        ####### Maximum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.maximize(x)
        o.check()
        #print(o.model()[x])

        max_x_range = int(str(o.model()[x]))
        
        
        for j in range(min_x_range, max_x_range):
            ####### Minimum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.minimize(y)
            o.check()

            min_y_range = o.model()[y]
            if min_y_range == None:
                min_y_range = 0

            ####### Maximum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.maximize(y)
            o.check()

            max_y_range = o.model()[y]
            if max_y_range == None:
                max_y_range = 0
                
            list_time_min[zone_id][j].append(int(str(min_y_range)))
            list_time_max[zone_id][j].append(int(str(max_y_range)))
    return list_time_min, list_time_max

# Attack Scheduling Function

In [14]:
def attack_scheduling(list_cluster, num_timeslots, start_time, final_time):
    list_time_min, list_time_max = range_calculation(list_cluster)
    num_timeslots = interval = 10

    prev_stay = 1
    prev_schedule = -1
    ultimate_cost = 0
    final_schedule = schedule = np.zeros((final_time - start_time))

    for init_time in range(start_time, final_time, interval):
        cost = [Int( 'cost_' + str(i)) for i in range(num_zones)]
        zones = [Int( 'zones_' + str(i)) for i in range(num_zones)]

        schedule = [Int( 'schedule_' + str(i)) for i in range(init_time, init_time + interval)]
        stay = [Int( 'stay_' + str(i)) for i in range(num_timeslots)]
        slot_cost = [Int( 'slot_cost_' + str(i)) for i in range(num_timeslots)]

        total_cost = Int('total_cost')

        o = Optimize()
        o.add(cost[0] == 0)
        o.add(cost[1] == 1)
        o.add(cost[2] == 2)
        o.add(cost[3] == 4)
        o.add(cost[4] == 3)


        ############################################################################
        ################## schedule should be withing a valid zone #################
        ############################################################################
        for t in range(len(schedule)):
            or_constraints = []
            for z in range(num_zones):
                or_constraints.append(schedule[t] == z)
            o.add(Or(or_constraints))

        ###############################################################################################################
        ################## if zone stay threshdold in current time is 0, do not schedule to that zone #################
        ###############################################################################################################

        # base case
        for z in range(1, num_zones):
            if list_time_min[z][init_time] == [] or (len(list_time_max[z][init_time]) == 1 and list_time_max[z][init_time][0] == 0):
                o.add(Implies(schedule[0] != prev_schedule, schedule[t] != z))

        for t in range(1, len(schedule)):
            for z in range(1, num_zones):
                if list_time_min[z][init_time + t] == [] or (len(list_time_max[z][init_time + t]) == 1 and list_time_max[z][init_time + t][0] == 0):
                    o.add(Implies(schedule[t] != schedule[t - 1], schedule[t] != z))

        #######################################################################################################################
        ############################################ constraints of stay ######################################################
        #######################################################################################################################
        ######## base case for time 0 ############
        if init_time == 0:
            o.add(stay[0] == 1)
        else:
            o.add(Implies(schedule[0] == prev_schedule, stay[0] == prev_stay + 1))
            o.add(Implies(Not(schedule[0] == prev_schedule), stay[0] == 1))

        for t in range(1, len(schedule)):
            continue_staying = (schedule[t] == schedule[t - 1])
            increment_stay = (stay[t] == stay[t - 1] + 1)
            reset_stay = (stay[t] == 1)

            o.add(Implies(continue_staying, increment_stay))
            o.add(Implies(Not(continue_staying), reset_stay))

        #######################################################################################################################
        ############ move to a zone different that previous timeslot if stay > max threshold in previous timeslot #############
        #######################################################################################################################
        ######## base case for time 0 ############
        o.add(Implies(prev_stay == max(list_time_max[prev_schedule][init_time - prev_stay], default=0), schedule[0] != prev_schedule))

        for t in range(1, len(schedule)):
            max_stay_threshold = 0
            for z in range(1, num_zones):
                for p_t in range(1, init_time + len(schedule)):
                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, p_t == max(list_time_max[z][init_time + t - p_t], default=0)), Not(continue_staying)))

        #######################################################################################################################
        # must stay in the zone same as the previous timeslot if stay < max && stay is in previous timeslot is out of cluster #
        #######################################################################################################################
        ######## base case for time 0 ############
        ranges_stay_constraints = []
        for k in range(len(list_time_min[prev_schedule][init_time - prev_stay])):
            ranges_stay_constraints.append(And(prev_stay >= list_time_min[prev_schedule][init_time - prev_stay][k], prev_stay <= list_time_max[prev_schedule][init_time - prev_stay][k]))            

        if init_time != 0:
            o.add(Implies(Not(Or(ranges_stay_constraints)), schedule[0] == prev_schedule))

        for t in range(1, len(schedule)):
            for z in range(1, num_zones):
                for p_t in range(1, init_time + t + 1):
                    ranges_stay_constraints = []
                    for k in range(len(list_time_min[z][init_time + t - p_t])):
                        ranges_stay_constraints.append(And(p_t >= list_time_min[z][init_time + t - p_t][k], p_t <= list_time_max[z][init_time + t - p_t][k]))            

                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, Not(Or(ranges_stay_constraints))), continue_staying))

        for t in range(len(schedule)):
            for z in range(num_zones):
                o.add(Implies(schedule[t] == z, slot_cost[t] == cost[z]))

        o.add(total_cost == Sum(slot_cost))

        #o.add(schedule[0] == 3)

        o.maximize(total_cost)

        print(o.check())
        print(init_time, o.model()[total_cost])
        ultimate_cost += int(str(o.model()[total_cost]))

        print("Ulimate Cost", init_time, ultimate_cost)

        print("num_timeslot", num_timeslots, "optimal_cost", o.model()[total_cost])

        for t in range(num_timeslots):
            print("schedule_" + str(init_time + t), o.model()[schedule[t]])
            final_schedule[init_time + t] = int(str(o.model()[schedule[t]]))

        for t in range(num_timeslots):
            print("stay_" + str(init_time + t), o.model()[stay[t]])

        prev_schedule = int(str(o.model()[schedule[-1]]))
        prev_stay = int(str(o.model()[stay[-1]]))
    return final_schedule

# Greedy Scheduling Function

In [11]:
def greedy_schedule(list_cluster, num_timeslots):
    current_timeslot = 0
    final_timeslot = 0

    alternate_schedule = np.zeros((num_timeslots))
    list_time_min, list_time_max = range_calculation(list_cluster)
    while final_timeslot <= 1439:
        for zone in [3, 4, 2, 1, 0]:
            max_list = list_time_max[zone][current_timeslot]
            for m in range(len(max_list)):
                max_list[m] = int(str(max_list[m]))
            if max_list == []:
                if zone == 0:
                    print("Critical", current_timeslot)
                    final_timeslot = current_timeslot + 1
                    alternate_schedule[current_timeslot: final_timeslot] = 0
                    current_timeslot = final_timeslot
                continue
            max_val = max(max_list)
            if max_val == 0:
                final_timeslot = current_timeslot + 1
                alternate_schedule[current_timeslot: final_timeslot] = 0
                current_timeslot = final_timeslot
            if max_val != 0:
                final_timeslot = current_timeslot + max_val
                print("Here", current_timeslot, final_timeslot)
                alternate_schedule[current_timeslot: final_timeslot] = zone
                print("selected zone", zone)
                current_timeslot = final_timeslot
                break
    return alternate_schedule

# Variable Parameters

In [12]:
num_timeslots = interval = 10
start_time = 0
final_time = 1440

# House-A Occupant-1 Attack Schedule

In [15]:
schedule_house_A_occ_1 = attack_scheduling(clusters_house_A_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/House-A_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 40
Ulimate Cost 0 40
num_timeslot 10 optimal_cost 40
schedule_0 3
schedule_1 3
schedule_2 3
schedule_3 3
schedule_4 3
schedule_5 3
schedule_6 3
schedule_7 3
schedule_8 3
schedule_9 3
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 37
Ulimate Cost 10 77
num_timeslot 10 optimal_cost 37
schedule_10 3
schedule_11 3
schedule_12 3
schedule_13 1
schedule_14 3
schedule_15 3
schedule_16 3
schedule_17 3
schedule_18 3
schedule_19 3
stay_10 11
stay_11 12
stay_12 13
stay_13 1
stay_14 1
stay_15 2
stay_16 3
stay_17 4
stay_18 5
stay_19 6
sat
20 22
Ulimate Cost 20 99
num_timeslot 10 optimal_cost 22
schedule_20 3
schedule_21 2
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 2
stay_20 7
stay_21 1
stay_22 2
stay_23 3
stay_24 4
stay_25 5
stay_26 6
stay_27 7
stay_28 8
stay_29 9
sat
30 24
Ulimate Cost 30 123
num_timeslot 10 optimal_cost 24
schedule_30 2
schedule_31 2
schedule_32 2
schedule_33

sat
260 0
Ulimate Cost 260 480
num_timeslot 10 optimal_cost 0
schedule_260 0
schedule_261 0
schedule_262 0
schedule_263 0
schedule_264 0
schedule_265 0
schedule_266 0
schedule_267 0
schedule_268 0
schedule_269 0
stay_260 91
stay_261 92
stay_262 93
stay_263 94
stay_264 95
stay_265 96
stay_266 97
stay_267 98
stay_268 99
stay_269 100
sat
270 0
Ulimate Cost 270 480
num_timeslot 10 optimal_cost 0
schedule_270 0
schedule_271 0
schedule_272 0
schedule_273 0
schedule_274 0
schedule_275 0
schedule_276 0
schedule_277 0
schedule_278 0
schedule_279 0
stay_270 101
stay_271 102
stay_272 103
stay_273 104
stay_274 105
stay_275 106
stay_276 107
stay_277 108
stay_278 109
stay_279 110
sat
280 0
Ulimate Cost 280 480
num_timeslot 10 optimal_cost 0
schedule_280 0
schedule_281 0
schedule_282 0
schedule_283 0
schedule_284 0
schedule_285 0
schedule_286 0
schedule_287 0
schedule_288 0
schedule_289 0
stay_280 111
stay_281 112
stay_282 113
stay_283 114
stay_284 115
stay_285 116
stay_286 117
stay_287 118
stay_288 

sat
500 0
Ulimate Cost 500 480
num_timeslot 10 optimal_cost 0
schedule_500 0
schedule_501 0
schedule_502 0
schedule_503 0
schedule_504 0
schedule_505 0
schedule_506 0
schedule_507 0
schedule_508 0
schedule_509 0
stay_500 331
stay_501 332
stay_502 333
stay_503 334
stay_504 335
stay_505 336
stay_506 337
stay_507 338
stay_508 339
stay_509 340
sat
510 0
Ulimate Cost 510 480
num_timeslot 10 optimal_cost 0
schedule_510 0
schedule_511 0
schedule_512 0
schedule_513 0
schedule_514 0
schedule_515 0
schedule_516 0
schedule_517 0
schedule_518 0
schedule_519 0
stay_510 341
stay_511 342
stay_512 343
stay_513 344
stay_514 345
stay_515 346
stay_516 347
stay_517 348
stay_518 349
stay_519 350
sat
520 0
Ulimate Cost 520 480
num_timeslot 10 optimal_cost 0
schedule_520 0
schedule_521 0
schedule_522 0
schedule_523 0
schedule_524 0
schedule_525 0
schedule_526 0
schedule_527 0
schedule_528 0
schedule_529 0
stay_520 351
stay_521 352
stay_522 353
stay_523 354
stay_524 355
stay_525 356
stay_526 357
stay_527 358


sat
750 32
Ulimate Cost 750 1280
num_timeslot 10 optimal_cost 32
schedule_750 3
schedule_751 3
schedule_752 4
schedule_753 4
schedule_754 4
schedule_755 4
schedule_756 4
schedule_757 4
schedule_758 4
schedule_759 4
stay_750 8
stay_751 9
stay_752 1
stay_753 2
stay_754 3
stay_755 4
stay_756 5
stay_757 6
stay_758 7
stay_759 8
sat
760 19
Ulimate Cost 760 1299
num_timeslot 10 optimal_cost 19
schedule_760 0
schedule_761 4
schedule_762 4
schedule_763 4
schedule_764 4
schedule_765 4
schedule_766 1
schedule_767 1
schedule_768 1
schedule_769 1
stay_760 1
stay_761 1
stay_762 2
stay_763 3
stay_764 4
stay_765 5
stay_766 1
stay_767 2
stay_768 3
stay_769 4
sat
770 10
Ulimate Cost 770 1309
num_timeslot 10 optimal_cost 10
schedule_770 1
schedule_771 1
schedule_772 1
schedule_773 1
schedule_774 1
schedule_775 1
schedule_776 1
schedule_777 1
schedule_778 1
schedule_779 1
stay_770 5
stay_771 6
stay_772 7
stay_773 8
stay_774 9
stay_775 10
stay_776 11
stay_777 12
stay_778 13
stay_779 14
sat
780 15
Ulimate C

sat
1000 40
Ulimate Cost 1000 2077
num_timeslot 10 optimal_cost 40
schedule_1000 3
schedule_1001 3
schedule_1002 3
schedule_1003 3
schedule_1004 3
schedule_1005 3
schedule_1006 3
schedule_1007 3
schedule_1008 3
schedule_1009 3
stay_1000 2
stay_1001 3
stay_1002 4
stay_1003 5
stay_1004 6
stay_1005 7
stay_1006 8
stay_1007 9
stay_1008 10
stay_1009 11
sat
1010 40
Ulimate Cost 1010 2117
num_timeslot 10 optimal_cost 40
schedule_1010 3
schedule_1011 3
schedule_1012 3
schedule_1013 3
schedule_1014 3
schedule_1015 3
schedule_1016 3
schedule_1017 3
schedule_1018 3
schedule_1019 3
stay_1010 12
stay_1011 13
stay_1012 14
stay_1013 15
stay_1014 16
stay_1015 17
stay_1016 18
stay_1017 19
stay_1018 20
stay_1019 21
sat
1020 40
Ulimate Cost 1020 2157
num_timeslot 10 optimal_cost 40
schedule_1020 3
schedule_1021 3
schedule_1022 3
schedule_1023 3
schedule_1024 3
schedule_1025 3
schedule_1026 3
schedule_1027 3
schedule_1028 3
schedule_1029 3
stay_1020 22
stay_1021 23
stay_1022 24
stay_1023 25
stay_1024 26
st

sat
1240 40
Ulimate Cost 1240 2900
num_timeslot 10 optimal_cost 40
schedule_1240 3
schedule_1241 3
schedule_1242 3
schedule_1243 3
schedule_1244 3
schedule_1245 3
schedule_1246 3
schedule_1247 3
schedule_1248 3
schedule_1249 3
stay_1240 43
stay_1241 44
stay_1242 45
stay_1243 46
stay_1244 47
stay_1245 48
stay_1246 49
stay_1247 50
stay_1248 51
stay_1249 52
sat
1250 40
Ulimate Cost 1250 2940
num_timeslot 10 optimal_cost 40
schedule_1250 3
schedule_1251 3
schedule_1252 3
schedule_1253 3
schedule_1254 3
schedule_1255 3
schedule_1256 3
schedule_1257 3
schedule_1258 3
schedule_1259 3
stay_1250 53
stay_1251 54
stay_1252 55
stay_1253 56
stay_1254 57
stay_1255 58
stay_1256 59
stay_1257 60
stay_1258 61
stay_1259 62
sat
1260 40
Ulimate Cost 1260 2980
num_timeslot 10 optimal_cost 40
schedule_1260 3
schedule_1261 3
schedule_1262 3
schedule_1263 3
schedule_1264 3
schedule_1265 3
schedule_1266 3
schedule_1267 3
schedule_1268 3
schedule_1269 3
stay_1260 63
stay_1261 64
stay_1262 65
stay_1263 66
stay_12

# House-A Occupant-1 Greedy Schedule

In [20]:
greedy_schedule_house_A_occ_1 = greedy_schedule(clusters_house_A_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/House-A_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 18
selected zone 3
Here 18 22
selected zone 3
Here 22 90
selected zone 2
Here 90 105
selected zone 4
Here 105 118
selected zone 4
Here 118 130
selected zone 4
Here 130 140
selected zone 4
Here 140 149
selected zone 4
Here 149 157
selected zone 4
Here 157 164
selected zone 4
Here 164 171
selected zone 4
Critical 171
Critical 172
Critical 173
Critical 174
Critical 175
Critical 176
Critical 177
Critical 178
Critical 179
Critical 180
Critical 181
Critical 182
Critical 183
Critical 184
Critical 185
Critical 186
Critical 187
Critical 188
Critical 189
Critical 190
Critical 191
Critical 192
Critical 193
Critical 194
Critical 195
Critical 196
Critical 197
Critical 198
Critical 199
Critical 200
Critical 201
Critical 202
Critical 203
Critical 204
Critical 205
Critical 206
Critical 207
Critical 208
Critical 209
Critical 210
Critical 211
Critical 212
Critical 213
Critical 214
Critical 215
Critical 216
Critical 217
Critical 218
Critical 219
Critical 220
Critical 221
Critical 222
Critical 223


# House-A Occupant-2 Attack Schedule

In [25]:
schedule_house_A_occ_2 = attack_scheduling(clusters_house_A_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/House-A_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 32
Ulimate Cost 0 32
num_timeslot 10 optimal_cost 32
schedule_0 3
schedule_1 0
schedule_2 3
schedule_3 3
schedule_4 0
schedule_5 3
schedule_6 3
schedule_7 3
schedule_8 3
schedule_9 3
stay_0 1
stay_1 1
stay_2 1
stay_3 2
stay_4 1
stay_5 1
stay_6 2
stay_7 3
stay_8 4
stay_9 5
sat
10 36
Ulimate Cost 10 68
num_timeslot 10 optimal_cost 36
schedule_10 0
schedule_11 3
schedule_12 3
schedule_13 3
schedule_14 3
schedule_15 3
schedule_16 3
schedule_17 3
schedule_18 3
schedule_19 3
stay_10 1
stay_11 1
stay_12 2
stay_13 3
stay_14 4
stay_15 5
stay_16 6
stay_17 7
stay_18 8
stay_19 9
sat
20 36
Ulimate Cost 20 104
num_timeslot 10 optimal_cost 36
schedule_20 3
schedule_21 3
schedule_22 0
schedule_23 3
schedule_24 3
schedule_25 3
schedule_26 3
schedule_27 3
schedule_28 3
schedule_29 3
stay_20 10
stay_21 11
stay_22 1
stay_23 1
stay_24 2
stay_25 3
stay_26 4
stay_27 5
stay_28 6
stay_29 7
sat
30 40
Ulimate Cost 30 144
num_timeslot 10 optimal_cost 40
schedule_30 3
schedule_31 3
schedule_32 3
schedule_33 

sat
260 0
Ulimate Cost 260 676
num_timeslot 10 optimal_cost 0
schedule_260 0
schedule_261 0
schedule_262 0
schedule_263 0
schedule_264 0
schedule_265 0
schedule_266 0
schedule_267 0
schedule_268 0
schedule_269 0
stay_260 82
stay_261 83
stay_262 84
stay_263 85
stay_264 86
stay_265 87
stay_266 88
stay_267 89
stay_268 90
stay_269 91
sat
270 0
Ulimate Cost 270 676
num_timeslot 10 optimal_cost 0
schedule_270 0
schedule_271 0
schedule_272 0
schedule_273 0
schedule_274 0
schedule_275 0
schedule_276 0
schedule_277 0
schedule_278 0
schedule_279 0
stay_270 92
stay_271 93
stay_272 94
stay_273 95
stay_274 96
stay_275 97
stay_276 98
stay_277 99
stay_278 100
stay_279 101
sat
280 0
Ulimate Cost 280 676
num_timeslot 10 optimal_cost 0
schedule_280 0
schedule_281 0
schedule_282 0
schedule_283 0
schedule_284 0
schedule_285 0
schedule_286 0
schedule_287 0
schedule_288 0
schedule_289 0
stay_280 102
stay_281 103
stay_282 104
stay_283 105
stay_284 106
stay_285 107
stay_286 108
stay_287 109
stay_288 110
stay_

sat
510 28
Ulimate Cost 510 1085
num_timeslot 10 optimal_cost 28
schedule_510 4
schedule_511 4
schedule_512 4
schedule_513 4
schedule_514 4
schedule_515 4
schedule_516 4
schedule_517 4
schedule_518 4
schedule_519 1
stay_510 29
stay_511 30
stay_512 31
stay_513 32
stay_514 33
stay_515 34
stay_516 35
stay_517 36
stay_518 37
stay_519 1
sat
520 10
Ulimate Cost 520 1095
num_timeslot 10 optimal_cost 10
schedule_520 1
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 2
stay_521 3
stay_522 4
stay_523 5
stay_524 6
stay_525 7
stay_526 8
stay_527 9
stay_528 10
stay_529 11
sat
530 9
Ulimate Cost 530 1104
num_timeslot 10 optimal_cost 9
schedule_530 1
schedule_531 1
schedule_532 1
schedule_533 0
schedule_534 1
schedule_535 1
schedule_536 1
schedule_537 1
schedule_538 1
schedule_539 1
stay_530 12
stay_531 13
stay_532 14
stay_533 1
stay_534 1
stay_535 2
stay_536 3
stay_537 4
stay_538 5
stay_539 6
sat
540 12
Ul

sat
760 0
Ulimate Cost 760 1278
num_timeslot 10 optimal_cost 0
schedule_760 0
schedule_761 0
schedule_762 0
schedule_763 0
schedule_764 0
schedule_765 0
schedule_766 0
schedule_767 0
schedule_768 0
schedule_769 0
stay_760 145
stay_761 146
stay_762 147
stay_763 148
stay_764 149
stay_765 150
stay_766 151
stay_767 152
stay_768 153
stay_769 154
sat
770 0
Ulimate Cost 770 1278
num_timeslot 10 optimal_cost 0
schedule_770 0
schedule_771 0
schedule_772 0
schedule_773 0
schedule_774 0
schedule_775 0
schedule_776 0
schedule_777 0
schedule_778 0
schedule_779 0
stay_770 155
stay_771 156
stay_772 157
stay_773 158
stay_774 159
stay_775 160
stay_776 161
stay_777 162
stay_778 163
stay_779 164
sat
780 0
Ulimate Cost 780 1278
num_timeslot 10 optimal_cost 0
schedule_780 0
schedule_781 0
schedule_782 0
schedule_783 0
schedule_784 0
schedule_785 0
schedule_786 0
schedule_787 0
schedule_788 0
schedule_789 0
stay_780 165
stay_781 166
stay_782 167
stay_783 168
stay_784 169
stay_785 170
stay_786 171
stay_787 1

sat
1010 18
Ulimate Cost 1010 1380
num_timeslot 10 optimal_cost 18
schedule_1010 2
schedule_1011 2
schedule_1012 0
schedule_1013 2
schedule_1014 2
schedule_1015 2
schedule_1016 2
schedule_1017 2
schedule_1018 2
schedule_1019 2
stay_1010 15
stay_1011 16
stay_1012 1
stay_1013 1
stay_1014 2
stay_1015 3
stay_1016 4
stay_1017 5
stay_1018 6
stay_1019 7
sat
1020 20
Ulimate Cost 1020 1400
num_timeslot 10 optimal_cost 20
schedule_1020 2
schedule_1021 2
schedule_1022 2
schedule_1023 2
schedule_1024 2
schedule_1025 2
schedule_1026 2
schedule_1027 2
schedule_1028 2
schedule_1029 2
stay_1020 8
stay_1021 9
stay_1022 10
stay_1023 11
stay_1024 12
stay_1025 13
stay_1026 14
stay_1027 15
stay_1028 16
stay_1029 17
sat
1030 18
Ulimate Cost 1030 1418
num_timeslot 10 optimal_cost 18
schedule_1030 2
schedule_1031 0
schedule_1032 2
schedule_1033 2
schedule_1034 2
schedule_1035 2
schedule_1036 2
schedule_1037 2
schedule_1038 2
schedule_1039 2
stay_1030 18
stay_1031 1
stay_1032 1
stay_1033 2
stay_1034 3
stay_103

sat
1250 30
Ulimate Cost 1250 1988
num_timeslot 10 optimal_cost 30
schedule_1250 3
schedule_1251 3
schedule_1252 3
schedule_1253 0
schedule_1254 4
schedule_1255 4
schedule_1256 4
schedule_1257 4
schedule_1258 4
schedule_1259 4
stay_1250 22
stay_1251 23
stay_1252 24
stay_1253 1
stay_1254 1
stay_1255 2
stay_1256 3
stay_1257 4
stay_1258 5
stay_1259 6
sat
1260 14
Ulimate Cost 1260 2002
num_timeslot 10 optimal_cost 14
schedule_1260 0
schedule_1261 0
schedule_1262 2
schedule_1263 2
schedule_1264 2
schedule_1265 2
schedule_1266 2
schedule_1267 2
schedule_1268 2
schedule_1269 0
stay_1260 1
stay_1261 2
stay_1262 1
stay_1263 2
stay_1264 3
stay_1265 4
stay_1266 5
stay_1267 6
stay_1268 7
stay_1269 1
sat
1270 18
Ulimate Cost 1270 2020
num_timeslot 10 optimal_cost 18
schedule_1270 0
schedule_1271 2
schedule_1272 2
schedule_1273 2
schedule_1274 2
schedule_1275 2
schedule_1276 2
schedule_1277 2
schedule_1278 2
schedule_1279 2
stay_1270 2
stay_1271 1
stay_1272 2
stay_1273 3
stay_1274 4
stay_1275 5
stay

# House-A Occupant-2 Greedy Schedule

In [26]:
greedy_schedule_house_A_occ_2 = greedy_schedule(clusters_house_A_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/House-A_Occ-2_Greedy_Schedule.csv", index = False)

Here 0 1
selected zone 3
Here 2 23
selected zone 2
Here 23 46
selected zone 3
Here 46 82
selected zone 3
Here 82 120
selected zone 3
Here 120 160
selected zone 3
Here 160 179
selected zone 4
Critical 179
Critical 180
Critical 181
Critical 182
Critical 183
Critical 184
Critical 185
Critical 186
Critical 187
Critical 188
Critical 189
Critical 190
Critical 191
Critical 192
Critical 193
Critical 194
Critical 195
Critical 196
Critical 197
Critical 198
Critical 199
Critical 200
Critical 201
Critical 202
Critical 203
Critical 204
Critical 205
Critical 206
Critical 207
Critical 208
Critical 209
Critical 210
Critical 211
Critical 212
Critical 213
Critical 214
Critical 215
Critical 216
Critical 217
Critical 218
Critical 219
Critical 220
Critical 221
Critical 222
Critical 223
Critical 224
Critical 225
Critical 226
Critical 227
Critical 228
Critical 229
Critical 230
Critical 231
Critical 232
Critical 233
Critical 234
Critical 235
Critical 236
Critical 237
Critical 238
Critical 239
Critical 240
Cri

# House-B Occupant-1 Attack Schedule

In [27]:
schedule_house_B_occ_1 = attack_scheduling(clusters_house_B_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/House-B_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 15
Ulimate Cost 0 15
num_timeslot 10 optimal_cost 15
schedule_0 0
schedule_1 0
schedule_2 0
schedule_3 4
schedule_4 4
schedule_5 4
schedule_6 0
schedule_7 4
schedule_8 4
schedule_9 0
stay_0 1
stay_1 2
stay_2 3
stay_3 1
stay_4 2
stay_5 3
stay_6 1
stay_7 1
stay_8 2
stay_9 1
sat
10 19
Ulimate Cost 10 34
num_timeslot 10 optimal_cost 19
schedule_10 0
schedule_11 4
schedule_12 4
schedule_13 0
schedule_14 4
schedule_15 4
schedule_16 0
schedule_17 4
schedule_18 4
schedule_19 1
stay_10 2
stay_11 1
stay_12 2
stay_13 1
stay_14 1
stay_15 2
stay_16 1
stay_17 1
stay_18 2
stay_19 1
sat
20 10
Ulimate Cost 20 44
num_timeslot 10 optimal_cost 10
schedule_20 1
schedule_21 1
schedule_22 1
schedule_23 1
schedule_24 1
schedule_25 1
schedule_26 1
schedule_27 1
schedule_28 1
schedule_29 1
stay_20 2
stay_21 3
stay_22 4
stay_23 5
stay_24 6
stay_25 7
stay_26 8
stay_27 9
stay_28 10
stay_29 11
sat
30 10
Ulimate Cost 30 54
num_timeslot 10 optimal_cost 10
schedule_30 1
schedule_31 1
schedule_32 1
schedule_33 1


sat
250 10
Ulimate Cost 250 274
num_timeslot 10 optimal_cost 10
schedule_250 1
schedule_251 1
schedule_252 1
schedule_253 1
schedule_254 1
schedule_255 1
schedule_256 1
schedule_257 1
schedule_258 1
schedule_259 1
stay_250 232
stay_251 233
stay_252 234
stay_253 235
stay_254 236
stay_255 237
stay_256 238
stay_257 239
stay_258 240
stay_259 241
sat
260 10
Ulimate Cost 260 284
num_timeslot 10 optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 242
stay_261 243
stay_262 244
stay_263 245
stay_264 246
stay_265 247
stay_266 248
stay_267 249
stay_268 250
stay_269 251
sat
270 10
Ulimate Cost 270 294
num_timeslot 10 optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 252
stay_271 253
stay_272 254
stay_273 255
stay_274 256
stay_275 257
stay_276 258
stay_27

sat
490 40
Ulimate Cost 490 598
num_timeslot 10 optimal_cost 40
schedule_490 3
schedule_491 3
schedule_492 3
schedule_493 3
schedule_494 3
schedule_495 3
schedule_496 3
schedule_497 3
schedule_498 3
schedule_499 3
stay_490 17
stay_491 18
stay_492 19
stay_493 20
stay_494 21
stay_495 22
stay_496 23
stay_497 24
stay_498 25
stay_499 26
sat
500 39
Ulimate Cost 500 637
num_timeslot 10 optimal_cost 39
schedule_500 4
schedule_501 3
schedule_502 3
schedule_503 3
schedule_504 3
schedule_505 3
schedule_506 3
schedule_507 3
schedule_508 3
schedule_509 3
stay_500 1
stay_501 1
stay_502 2
stay_503 3
stay_504 4
stay_505 5
stay_506 6
stay_507 7
stay_508 8
stay_509 9
sat
510 40
Ulimate Cost 510 677
num_timeslot 10 optimal_cost 40
schedule_510 3
schedule_511 3
schedule_512 3
schedule_513 3
schedule_514 3
schedule_515 3
schedule_516 3
schedule_517 3
schedule_518 3
schedule_519 3
stay_510 10
stay_511 11
stay_512 12
stay_513 13
stay_514 14
stay_515 15
stay_516 16
stay_517 17
stay_518 18
stay_519 19
sat
520 

sat
740 18
Ulimate Cost 740 1535
num_timeslot 10 optimal_cost 18
schedule_740 0
schedule_741 4
schedule_742 4
schedule_743 4
schedule_744 0
schedule_745 4
schedule_746 4
schedule_747 0
schedule_748 4
schedule_749 0
stay_740 1
stay_741 1
stay_742 2
stay_743 3
stay_744 1
stay_745 1
stay_746 2
stay_747 1
stay_748 1
stay_749 1
sat
750 18
Ulimate Cost 750 1553
num_timeslot 10 optimal_cost 18
schedule_750 0
schedule_751 2
schedule_752 2
schedule_753 2
schedule_754 2
schedule_755 2
schedule_756 2
schedule_757 2
schedule_758 2
schedule_759 2
stay_750 2
stay_751 1
stay_752 2
stay_753 3
stay_754 4
stay_755 5
stay_756 6
stay_757 7
stay_758 8
stay_759 9
sat
760 20
Ulimate Cost 760 1573
num_timeslot 10 optimal_cost 20
schedule_760 2
schedule_761 2
schedule_762 2
schedule_763 2
schedule_764 2
schedule_765 2
schedule_766 2
schedule_767 2
schedule_768 2
schedule_769 2
stay_760 10
stay_761 11
stay_762 12
stay_763 13
stay_764 14
stay_765 15
stay_766 16
stay_767 17
stay_768 18
stay_769 19
sat
770 20
Ulim

sat
990 27
Ulimate Cost 990 2245
num_timeslot 10 optimal_cost 27
schedule_990 4
schedule_991 0
schedule_992 4
schedule_993 4
schedule_994 4
schedule_995 4
schedule_996 4
schedule_997 4
schedule_998 4
schedule_999 4
stay_990 7
stay_991 1
stay_992 1
stay_993 2
stay_994 3
stay_995 4
stay_996 5
stay_997 6
stay_998 7
stay_999 8
sat
1000 27
Ulimate Cost 1000 2272
num_timeslot 10 optimal_cost 27
schedule_1000 0
schedule_1001 4
schedule_1002 4
schedule_1003 4
schedule_1004 4
schedule_1005 4
schedule_1006 4
schedule_1007 4
schedule_1008 4
schedule_1009 4
stay_1000 1
stay_1001 1
stay_1002 2
stay_1003 3
stay_1004 4
stay_1005 5
stay_1006 6
stay_1007 7
stay_1008 8
stay_1009 9
sat
1010 3
Ulimate Cost 1010 2275
num_timeslot 10 optimal_cost 3
schedule_1010 4
schedule_1011 0
schedule_1012 0
schedule_1013 0
schedule_1014 0
schedule_1015 0
schedule_1016 0
schedule_1017 0
schedule_1018 0
schedule_1019 0
stay_1010 10
stay_1011 1
stay_1012 2
stay_1013 3
stay_1014 4
stay_1015 5
stay_1016 6
stay_1017 7
stay_1

sat
1230 39
Ulimate Cost 1230 2840
num_timeslot 10 optimal_cost 39
schedule_1230 3
schedule_1231 3
schedule_1232 3
schedule_1233 3
schedule_1234 4
schedule_1235 3
schedule_1236 3
schedule_1237 3
schedule_1238 3
schedule_1239 3
stay_1230 11
stay_1231 12
stay_1232 13
stay_1233 14
stay_1234 1
stay_1235 1
stay_1236 2
stay_1237 3
stay_1238 4
stay_1239 5
sat
1240 40
Ulimate Cost 1240 2880
num_timeslot 10 optimal_cost 40
schedule_1240 3
schedule_1241 3
schedule_1242 3
schedule_1243 3
schedule_1244 3
schedule_1245 3
schedule_1246 3
schedule_1247 3
schedule_1248 3
schedule_1249 3
stay_1240 6
stay_1241 7
stay_1242 8
stay_1243 9
stay_1244 10
stay_1245 11
stay_1246 12
stay_1247 13
stay_1248 14
stay_1249 15
sat
1250 39
Ulimate Cost 1250 2919
num_timeslot 10 optimal_cost 39
schedule_1250 3
schedule_1251 3
schedule_1252 3
schedule_1253 3
schedule_1254 4
schedule_1255 3
schedule_1256 3
schedule_1257 3
schedule_1258 3
schedule_1259 3
stay_1250 16
stay_1251 17
stay_1252 18
stay_1253 19
stay_1254 1
stay_

# House-B Occupant-1 Greedy Schedule

In [28]:
greedy_schedule_house_B_occ_1 = greedy_schedule(clusters_house_B_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/House-B_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 611
selected zone 1
Here 611 642
selected zone 3
Here 642 674
selected zone 3
Here 674 706
selected zone 3
Here 706 717
selected zone 3
Here 717 725
selected zone 4
Here 725 732
selected zone 4
Here 732 737
selected zone 4
Here 737 741
selected zone 4
Here 741 744
selected zone 4
Here 744 746
selected zone 4
Here 746 747
selected zone 4
Here 747 748
selected zone 4
Here 748 749
selected zone 4
Critical 749
Critical 750
Here 751 786
selected zone 2
Here 786 794
selected zone 4
Here 794 803
selected zone 4
Here 803 811
selected zone 4
Here 811 832
selected zone 3
Here 832 864
selected zone 3
Critical 864
Critical 865
Critical 866
Critical 867
Critical 868
Here 869 874
selected zone 2
Here 874 888
selected zone 2
Critical 888
Critical 889
Critical 890
Critical 891
Critical 892
Critical 893
Critical 894
Critical 895
Critical 896
Critical 897
Critical 898
Critical 899
Critical 900
Critical 901
Critical 902
Critical 904
Here 905 906
selected zone 4
Here 906 907
selected zone 4
Here 90

# House-B Occupant-2 Attack Schedule

In [29]:
schedule_house_B_occ_2 = attack_scheduling(clusters_house_B_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/House-B_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 10
Ulimate Cost 0 10
num_timeslot 10 optimal_cost 10
schedule_0 1
schedule_1 1
schedule_2 1
schedule_3 1
schedule_4 1
schedule_5 1
schedule_6 1
schedule_7 1
schedule_8 1
schedule_9 1
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 10
Ulimate Cost 10 20
num_timeslot 10 optimal_cost 10
schedule_10 1
schedule_11 1
schedule_12 1
schedule_13 1
schedule_14 1
schedule_15 1
schedule_16 1
schedule_17 1
schedule_18 1
schedule_19 1
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 17
stay_17 18
stay_18 19
stay_19 20
sat
20 10
Ulimate Cost 20 30
num_timeslot 10 optimal_cost 10
schedule_20 1
schedule_21 1
schedule_22 1
schedule_23 1
schedule_24 1
schedule_25 1
schedule_26 1
schedule_27 1
schedule_28 1
schedule_29 1
stay_20 21
stay_21 22
stay_22 23
stay_23 24
stay_24 25
stay_25 26
stay_26 27
stay_27 28
stay_28 29
stay_29 30
sat
30 10
Ulimate Cost 30 40
num_timeslot 10 optimal_cost 10
schedule_30 1
schedule_31 1
schedule_

sat
250 10
Ulimate Cost 250 260
num_timeslot 10 optimal_cost 10
schedule_250 1
schedule_251 1
schedule_252 1
schedule_253 1
schedule_254 1
schedule_255 1
schedule_256 1
schedule_257 1
schedule_258 1
schedule_259 1
stay_250 251
stay_251 252
stay_252 253
stay_253 254
stay_254 255
stay_255 256
stay_256 257
stay_257 258
stay_258 259
stay_259 260
sat
260 10
Ulimate Cost 260 270
num_timeslot 10 optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 261
stay_261 262
stay_262 263
stay_263 264
stay_264 265
stay_265 266
stay_266 267
stay_267 268
stay_268 269
stay_269 270
sat
270 10
Ulimate Cost 270 280
num_timeslot 10 optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 271
stay_271 272
stay_272 273
stay_273 274
stay_274 275
stay_275 276
stay_276 277
stay_27

sat
500 39
Ulimate Cost 500 624
num_timeslot 10 optimal_cost 39
schedule_500 3
schedule_501 4
schedule_502 3
schedule_503 3
schedule_504 3
schedule_505 3
schedule_506 3
schedule_507 3
schedule_508 3
schedule_509 3
stay_500 12
stay_501 1
stay_502 1
stay_503 2
stay_504 3
stay_505 4
stay_506 5
stay_507 6
stay_508 7
stay_509 8
sat
510 31
Ulimate Cost 510 655
num_timeslot 10 optimal_cost 31
schedule_510 3
schedule_511 4
schedule_512 4
schedule_513 4
schedule_514 4
schedule_515 4
schedule_516 4
schedule_517 4
schedule_518 4
schedule_519 4
stay_510 9
stay_511 1
stay_512 2
stay_513 3
stay_514 4
stay_515 5
stay_516 6
stay_517 7
stay_518 8
stay_519 9
sat
520 27
Ulimate Cost 520 682
num_timeslot 10 optimal_cost 27
schedule_520 4
schedule_521 4
schedule_522 4
schedule_523 4
schedule_524 4
schedule_525 0
schedule_526 4
schedule_527 4
schedule_528 4
schedule_529 4
stay_520 10
stay_521 11
stay_522 12
stay_523 13
stay_524 14
stay_525 1
stay_526 1
stay_527 2
stay_528 3
stay_529 4
sat
530 33
Ulimate Cos

sat
760 30
Ulimate Cost 760 1452
num_timeslot 10 optimal_cost 30
schedule_760 4
schedule_761 4
schedule_762 4
schedule_763 4
schedule_764 4
schedule_765 4
schedule_766 4
schedule_767 4
schedule_768 4
schedule_769 4
stay_760 6
stay_761 7
stay_762 8
stay_763 9
stay_764 10
stay_765 11
stay_766 12
stay_767 13
stay_768 14
stay_769 15
sat
770 3
Ulimate Cost 770 1455
num_timeslot 10 optimal_cost 3
schedule_770 4
schedule_771 0
schedule_772 0
schedule_773 0
schedule_774 0
schedule_775 0
schedule_776 0
schedule_777 0
schedule_778 0
schedule_779 0
stay_770 16
stay_771 1
stay_772 2
stay_773 3
stay_774 4
stay_775 5
stay_776 6
stay_777 7
stay_778 8
stay_779 9
sat
780 0
Ulimate Cost 780 1455
num_timeslot 10 optimal_cost 0
schedule_780 0
schedule_781 0
schedule_782 0
schedule_783 0
schedule_784 0
schedule_785 0
schedule_786 0
schedule_787 0
schedule_788 0
schedule_789 0
stay_780 10
stay_781 11
stay_782 12
stay_783 13
stay_784 14
stay_785 15
stay_786 16
stay_787 17
stay_788 18
stay_789 19
sat
790 0
Ul

sat
1010 0
Ulimate Cost 1010 1455
num_timeslot 10 optimal_cost 0
schedule_1010 0
schedule_1011 0
schedule_1012 0
schedule_1013 0
schedule_1014 0
schedule_1015 0
schedule_1016 0
schedule_1017 0
schedule_1018 0
schedule_1019 0
stay_1010 240
stay_1011 241
stay_1012 242
stay_1013 243
stay_1014 244
stay_1015 245
stay_1016 246
stay_1017 247
stay_1018 248
stay_1019 249
sat
1020 0
Ulimate Cost 1020 1455
num_timeslot 10 optimal_cost 0
schedule_1020 0
schedule_1021 0
schedule_1022 0
schedule_1023 0
schedule_1024 0
schedule_1025 0
schedule_1026 0
schedule_1027 0
schedule_1028 0
schedule_1029 0
stay_1020 250
stay_1021 251
stay_1022 252
stay_1023 253
stay_1024 254
stay_1025 255
stay_1026 256
stay_1027 257
stay_1028 258
stay_1029 259
sat
1030 0
Ulimate Cost 1030 1455
num_timeslot 10 optimal_cost 0
schedule_1030 0
schedule_1031 0
schedule_1032 0
schedule_1033 0
schedule_1034 0
schedule_1035 0
schedule_1036 0
schedule_1037 0
schedule_1038 0
schedule_1039 0
stay_1030 260
stay_1031 261
stay_1032 262
sta

sat
1240 10
Ulimate Cost 1240 1508
num_timeslot 10 optimal_cost 10
schedule_1240 1
schedule_1241 1
schedule_1242 1
schedule_1243 1
schedule_1244 1
schedule_1245 1
schedule_1246 1
schedule_1247 1
schedule_1248 1
schedule_1249 1
stay_1240 1
stay_1241 2
stay_1242 3
stay_1243 4
stay_1244 5
stay_1245 6
stay_1246 7
stay_1247 8
stay_1248 9
stay_1249 10
sat
1250 10
Ulimate Cost 1250 1518
num_timeslot 10 optimal_cost 10
schedule_1250 1
schedule_1251 1
schedule_1252 1
schedule_1253 1
schedule_1254 1
schedule_1255 1
schedule_1256 1
schedule_1257 1
schedule_1258 1
schedule_1259 1
stay_1250 11
stay_1251 12
stay_1252 13
stay_1253 14
stay_1254 15
stay_1255 16
stay_1256 17
stay_1257 18
stay_1258 19
stay_1259 20
sat
1260 6
Ulimate Cost 1260 1524
num_timeslot 10 optimal_cost 6
schedule_1260 1
schedule_1261 1
schedule_1262 1
schedule_1263 1
schedule_1264 1
schedule_1265 1
schedule_1266 0
schedule_1267 0
schedule_1268 0
schedule_1269 0
stay_1260 21
stay_1261 22
stay_1262 23
stay_1263 24
stay_1264 25
stay_

# House-B Occupant-2 Greedy Schedule

In [30]:
greedy_schedule_house_B_occ_2 = greedy_schedule(clusters_house_B_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/House-B_Occ-2_Greedy_Schedule.csv", index = False)

Here 0 428
selected zone 1
Here 428 436
selected zone 4
Here 436 445
selected zone 4
Here 445 455
selected zone 4
Here 455 466
selected zone 4
Here 466 478
selected zone 4
Here 478 491
selected zone 4
Here 491 502
selected zone 3
Here 502 511
selected zone 3
Here 511 527
selected zone 4
Here 527 545
selected zone 4
Here 545 554
selected zone 3
Here 554 564
selected zone 3
Here 564 576
selected zone 3
Here 576 597
selected zone 4
Here 597 618
selected zone 4
Here 618 633
selected zone 3
Here 633 649
selected zone 3
Here 649 665
selected zone 3
Here 665 682
selected zone 3
Here 682 699
selected zone 3
Here 699 714
selected zone 3
Critical 714
Critical 715
Critical 716
Critical 717
Critical 718
Critical 719
Critical 720
Critical 721
Critical 722
Critical 723
Critical 724
Critical 725
Critical 726
Critical 727
Critical 728
Critical 729
Critical 730
Critical 731
Critical 732
Critical 733
Critical 734
Critical 735
Critical 736
Here 737 754
selected zone 4
Here 754 770
selected zone 4
Here 77

In [31]:
cost_arr = [0, 1, 2, 4, 3]

total_final_schedule = 0
total_alternate_schedule = 0
for i in range(len(schedule_house_A_occ_2)):
    total_final_schedule += cost_arr[int(schedule_house_A_occ_2[i])]
    total_alternate_schedule += cost_arr[int(greedy_schedule_house_A_occ_2[i])]
    print(i, "schedule", schedule_house_A_occ_2[i], "schedule cost", cost_arr[int(schedule_house_A_occ_2[i])], "alternate schedule", greedy_schedule_house_A_occ_2[i], "alternate schedule cost", cost_arr[int(greedy_schedule_house_A_occ_2[i])], total_final_schedule, total_alternate_schedule)

0 schedule 3.0 schedule cost 4 alternate schedule 3.0 alternate schedule cost 4 4 4
1 schedule 0.0 schedule cost 0 alternate schedule 0.0 alternate schedule cost 0 4 4
2 schedule 3.0 schedule cost 4 alternate schedule 2.0 alternate schedule cost 2 8 6
3 schedule 3.0 schedule cost 4 alternate schedule 2.0 alternate schedule cost 2 12 8
4 schedule 0.0 schedule cost 0 alternate schedule 2.0 alternate schedule cost 2 12 10
5 schedule 3.0 schedule cost 4 alternate schedule 2.0 alternate schedule cost 2 16 12
6 schedule 3.0 schedule cost 4 alternate schedule 2.0 alternate schedule cost 2 20 14
7 schedule 3.0 schedule cost 4 alternate schedule 2.0 alternate schedule cost 2 24 16
8 schedule 3.0 schedule cost 4 alternate schedule 2.0 alternate schedule cost 2 28 18
9 schedule 3.0 schedule cost 4 alternate schedule 2.0 alternate schedule cost 2 32 20
10 schedule 0.0 schedule cost 0 alternate schedule 2.0 alternate schedule cost 2 32 22
11 schedule 3.0 schedule cost 4 alternate schedule 2.0 alter